# CompLing Final Project- Automated Web Crawling

### In this notebook, we will learn a concept of the package-BeautifulSoup and its implementation

## Outline:

<img src="img/outline.png" alt="Outline" style="width:800px;"/>

## HTTP Protocol

<img src="img/http.png" alt="Http" style="width:600px;"/>

HTTP functions as a request-response protocol in the client-server computing model. The **client** submits an HTTP _request_ message to the server. The **server**, which provides resources such as HTML files and other contents, returns a _response_ message to the client.

## Scrape the titles and comments from the website-Reddit:


In [25]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import schedule
import time

'''url=the url of the website that I want to scrape,
html=use the urlopen function to fetch URLs
soup=use the beautifulsoup to parse the response from the server'''

url = "https://www.reddit.com/"
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

reddit_text = soup.find_all("h3",class_="_eYtD2XCVieq6emjKBH3m")
reddit_texts = [e.text for e in reddit_text]

reddit_comment = soup.find_all("span",class_= "FHCV02u6Cp2zYL0fhQPsO")
reddit_comments = [f.text for f in reddit_comment]

print(reddit_text)
print(reddit_comment)

[<h3 class="_eYtD2XCVieq6emjKBH3m">What is something in your search history that you are ashamed off and it's not porn or hentai of any kind?</h3>, <h3 class="_eYtD2XCVieq6emjKBH3m">AITA for refusing to support my son's pregnant girlfriend until she takes a paternity test?</h3>, <h3 class="_eYtD2XCVieq6emjKBH3m">me_irl</h3>, <h3 class="_eYtD2XCVieq6emjKBH3m">My Valkyrie Cosplay from God of War</h3>, <h3 class="_eYtD2XCVieq6emjKBH3m">Runner who slapped reporter's behind on live TV has been identified and banned from future running events, charges pending</h3>, <h3 class="_eYtD2XCVieq6emjKBH3m">US States ordered by life-expectancy - Animation [OC]</h3>, <h3 class="_eYtD2XCVieq6emjKBH3m">Forum - Development Manifesto - Development Manifesto: Balance in Path of Exile: Conquerors of the Atlas - Path of Exile</h3>]
[<span class="FHCV02u6Cp2zYL0fhQPsO">8.7k comments</span>, <span class="FHCV02u6Cp2zYL0fhQPsO">5.4k comments</span>, <span class="FHCV02u6Cp2zYL0fhQPsO">216 comments</span>, <span

In [26]:
print(reddit_texts)
print(reddit_comments)

["What is something in your search history that you are ashamed off and it's not porn or hentai of any kind?", "AITA for refusing to support my son's pregnant girlfriend until she takes a paternity test?", 'me_irl', 'My Valkyrie Cosplay from God of War', "Runner who slapped reporter's behind on live TV has been identified and banned from future running events, charges pending", 'US States ordered by life-expectancy - Animation [OC]', 'Forum - Development Manifesto - Development Manifesto: Balance in Path of Exile: Conquerors of the Atlas - Path of Exile']
['8.7k comments', '5.4k comments', '216 comments', '387 comments', '3.2k comments', '2.8k comments', '2.7k comments']


In [27]:
 '''clear all the punctuations and lowercase the texts in the title'''
    
a=0
for a in range(len(reddit_texts)):
    for b in '!"#$%&()*+,-./:;<=>?@[\\]^_‘{|}~':
        reddit_texts[a] = reddit_texts[a].replace(b, "")
        reddit_texts[a] = reddit_texts[a].lower()

print(reddit_texts)

["what is something in your search history that you are ashamed off and it's not porn or hentai of any kind", "aita for refusing to support my son's pregnant girlfriend until she takes a paternity test", 'meirl', 'my valkyrie cosplay from god of war', "runner who slapped reporter's behind on live tv has been identified and banned from future running events charges pending", 'us states ordered by lifeexpectancy  animation oc', 'forum  development manifesto  development manifesto balance in path of exile conquerors of the atlas  path of exile']


In [28]:
'''check if there is a repeated title'''
print(len(reddit_texts))
print(len(reddit_comments))

7
7


In [29]:
'''delete the repeated title if there is one'''
c=0
reddit_texts1=[]
for c in range(len(reddit_texts)):
    if reddit_texts[c] not in reddit_texts1:
        reddit_texts1.append(reddit_texts[c])

print(reddit_texts1)

["what is something in your search history that you are ashamed off and it's not porn or hentai of any kind", "aita for refusing to support my son's pregnant girlfriend until she takes a paternity test", 'meirl', 'my valkyrie cosplay from god of war', "runner who slapped reporter's behind on live tv has been identified and banned from future running events charges pending", 'us states ordered by lifeexpectancy  animation oc', 'forum  development manifesto  development manifesto balance in path of exile conquerors of the atlas  path of exile']


In [30]:
'''check again'''
print(len(reddit_texts1))
print(len(reddit_comments))

7
7


In [31]:
'''delete the word "comments"'''
d=0
for d in range(len(reddit_comments)):
    reddit_comments[d]=reddit_comments[d].replace("comments","")
    
print(reddit_comments)

['8.7k ', '5.4k ', '216 ', '387 ', '3.2k ', '2.8k ', '2.7k ']


In [33]:
'''search word from the titles, if it matches, append the titles that contain the search word into the 
new list reddit_search, and also append the corresponding comments to the new list reddit_comments'''
    
search_word=["history","development"]
e=0
reddit_search=[]
reddit_search_comment=[]
for e in range(len(reddit_texts1)):
    words = reddit_texts1[e].split()
    for word in words:
        if word in search_word and reddit_texts1[e] not in reddit_search:
            reddit_search.append(reddit_texts1[e])
            reddit_search_comment.append(reddit_comments[e])
    
print(reddit_search)
print(reddit_search_comment)

["what is something in your search history that you are ashamed off and it's not porn or hentai of any kind", 'forum  development manifesto  development manifesto balance in path of exile conquerors of the atlas  path of exile']
['8.7k ', '2.7k ']


In [34]:
'''delete the ".","k","", plus 2 zeros and convert the list to interger
if the number is too large, the format will become BLOB when storing in the database which is hard to read
so I divide the number by 10 to get the numeric format'''

f=0
for f in range(len(reddit_search_comment)):
    if "k" in reddit_search_comment[f]:
        reddit_search_comment[f]=reddit_search_comment[f].replace(".","")
        reddit_search_comment[f]=reddit_search_comment[f].replace("k","")
        reddit_search_comment[f]=reddit_search_comment[f].replace(" ","")
        reddit_search_comment[f]=reddit_search_comment[f]+"00"
        reddit_search_comment[f]=(int(reddit_search_comment[f])/10)
    else:
        reddit_search_comment[f]=(int(reddit_search_comment[f])/10)
        
        
print(reddit_search)
print(reddit_search_comment)

["what is something in your search history that you are ashamed off and it's not porn or hentai of any kind", 'forum  development manifesto  development manifesto balance in path of exile conquerors of the atlas  path of exile']
[870.0, 270.0]


In [35]:
'''use the panda package to convert to dataframe(table) format'''

df = pd.DataFrame(
    {"reddit_search":reddit_search, "reddit_search_comment":reddit_search_comment
    })
df.head()

,reddit_search,reddit_search_comment
0,what is something in your search history that ...,870.0
1,forum development manifesto development mani...,270.0


In [36]:
''' A new file called test.db is created where the database will be stored,
    connect Python to database, create the table in database'''

conn = sqlite3.connect('test.db')
c = conn.cursor()
#c.execute("CREATE TABLE COMPLING1206 (TITLE TEXT, COMMENTS INTEGER);"
    #)

<img src="img/cursor1.png" alt="cursor" style="width:600px;"/>

In [37]:
'''insert the data into the table'''

g=0
for g in range(len(df)):
    c.execute("INSERT INTO COMPLING1206 VALUES (?,?)", (df["reddit_search"][g],df["reddit_search_comment"][g],))
    conn.commit()

## Scheduler

In [38]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import URLError
import pandas as pd
import sqlite3
import schedule
import time

def redditsqlitegogo():
    try:
        url = "https://www.reddit.com/"
        html = urlopen(url)
        soup = BeautifulSoup(html, "html.parser")

        reddit_text = soup.find_all("h3",class_="_eYtD2XCVieq6emjKBH3m")
        reddit_texts = [e.text for e in reddit_text]

        reddit_comment = soup.find_all("span",class_= "FHCV02u6Cp2zYL0fhQPsO")
        reddit_comments = [f.text for f in reddit_comment]
    
        #print(reddit_comments)

       
        a=0
        for a in range(len(reddit_texts)):
            for b in '!"#$%&()*+,-./:;<=>?@[\\]^_‘{|}~':
                reddit_texts[a] = reddit_texts[a].replace(b, " ")
                reddit_texts[a] = reddit_texts[a].lower()
    
        #print(reddit_texts)   
        
        c=0
        reddit_texts1=[]
        for c in range(len(reddit_texts)):
            if reddit_texts[c] not in reddit_texts1:
                reddit_texts1.append(reddit_texts[c])
    
        print(reddit_texts)
        # When encountering more than two repeated titles, it's hard to match the titles and comments,
        #so I abandon the repeated titles for this time period (this happens rarely)
        h=0
        if len(reddit_texts1) != len(reddit_comments):
            reddit_texts1=[]
            while h < len(reddit_comments):
                reddit_texts1.append("No Value")
                h=h+1
        
        d=0
        for d in range(len(reddit_comments)):
            reddit_comments[d]=reddit_comments[d].replace("comments","")
        
        e=0
        for e in range(len(reddit_comments)):
            if "k" in reddit_comments[e]:
                reddit_comments[e]=reddit_comments[e].replace(".","")
                reddit_comments[e]=reddit_comments[e].replace("k","")
                reddit_comments[e]=reddit_comments[e].replace(" ","")
                reddit_comments[e]=reddit_comments[e]+"000"
                reddit_comments[e]=(int(reddit_comments[e])/100)
            else:
                reddit_comments[e]=(int(reddit_comments[e])/10)
            
       
        df = pd.DataFrame(
            {"reddit_texts1":reddit_texts1, "reddit_comments":reddit_comments
        })
        df.head()

        
        conn = sqlite3.connect('test.db')
        c = conn.cursor()
        #c.execute("CREATE TABLE REDDITTILECOMMENTS11145 (TITLE TEXT, COMMENTS INTEGER);"
        #)



        f=0
        for f in range(len(df)):
            c.execute("INSERT INTO COMPLING1206 VALUES (?,?)", (df["reddit_texts1"][f],df["reddit_comments"][f],))
            conn.commit()
            
    except URLError:
        print("Be patient~let's try again")

In [39]:
import schedule
import time

schedule.every(10).seconds.do(redditsqlitegogo)

while 1:
    schedule.run_pending()
    time.sleep(1)

Be patient~let's try again
Be patient~let's try again
Be patient~let's try again
Be patient~let's try again
Be patient~let's try again
Be patient~let's try again
Be patient~let's try again
["what is something in your search history that you are ashamed off and it's not porn or hentai of any kind ", "aita for refusing to support my son's pregnant girlfriend until she takes a paternity test ", 'me irl', 'my valkyrie cosplay from god of war', "runner who slapped reporter's behind on live tv has been identified and banned from future running events  charges pending", 'us states ordered by life expectancy   animation  oc ', 'forum   development manifesto   development manifesto  balance in path of exile  conquerors of the atlas   path of exile']
["what is something in your search history that you are ashamed off and it's not porn or hentai of any kind ", "aita for refusing to support my son's pregnant girlfriend until she takes a paternity test ", 'me irl', 'my valkyrie cosplay from god of 

KeyboardInterrupt: 

## Google Blog

In [40]:
'''scrape articles from GoogleBlog,'''
'''fetch google blog article, enter the next page and fetch the content of the article'''

from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
import sqlite3

url = "https://www.blog.google/outreach-initiatives/arts-culture/"
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

'''scrape the title of the next page (because the connection between the main page's url and the next page's url
is main page's url + names of the titles of the next page)'''

google_text = soup.find_all("h4",class_="uni-blog-nup__header h-has-bottom-margin h-u-font-weight-medium")
google_texts = [e.text for e in google_text]

'''lowercase the title of the article and join the "-" to become the part that comes at the very end of a URL'''
a=0
google_url=[]
for a in range(len(google_texts)):
    google_texts[a]=google_texts[a].lower()
    url=google_texts[a].split()
    google_url.append("-".join(url))
    

'''connect the main page's url and the next page'''

web="https://www.blog.google/outreach-initiatives/arts-culture/"
b=0
total_url=[]
for b in range(len(google_url)):
    total_url.append(web+google_url[b]+"/")
    
    
'''enter the next page to fetch the content of the article'''  

url = total_url[0]
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

google_new=soup.find_all("div",class_="rich-text")
google_news=[f.text for f in google_new]

print(google_news) 

['At the Google Arts & Culture Lab in Paris, we’re all about exploring the relationship between art and technology. Since 2012, we’ve worked with artists and creators from many fields, developing experiments that let you design patterns in augmented reality, co-create poetry, or experience multisensory art installations. Today we’re launching two experiments to test the potential of artificial intelligence in the worlds of contemporary dance and fashion.For our first experiment, Runway Palette, we came together with The Business of Fashion, whose collection includes 140,000 photos of runway looks from almost 4,000 fashion shows. If you could attend one fashion show per day, it would take you more than ten years to see them all. By extracting the main colors of each look, we used machine learning to organize the images by color palette, resulting in an interactive visualization of four years of fashion by almost 1,000 designers.Everyone can now use the color palette visualization to exp

## Twitter BBC

In [41]:
'''enter the next page to fetch the content of the article'''  

from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
import sqlite3


url = "https://twitter.com/BBCWorld"
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

tweet_text = soup.find_all("p",class_="tweet-text")
tweet_texts = [e.text for e in tweet_text]

'''the titles that I scraped have the url and some symbols after the titles, so here I delete them'''

new_tweet=[]
new_tweet2=[]
a=0
for a in range(len(tweet_texts)):
    tweet_texts[a] = tweet_texts[a].replace('"', " ")
    tweet_texts[a] = tweet_texts[a].replace("\n","")
    tweet_texts[a] = tweet_texts[a].replace("#","")
    new_tweet = re.split("https://",tweet_texts[a])
    del new_tweet[len(new_tweet)-1]
    new_tweet2.extend(new_tweet)
    
        
print(tweet_texts)
print(new_tweet2)

['Trump impeachment: House Democrats to unveil formal chargeshttps://bbc.in/2YygkMi\xa0', 'Floods and power cuts hit South Africahttps://bbc.in/2LDYWR0\xa0', 'Algeria jails two former prime ministers over corruptionhttps://bbc.in/35bKuY8\xa0', 'France protest: Mass rallies called on sixth day of disruptionhttps://bbc.in/356qjLe\xa0', 'New Zealand volcano: What we know about those affectedhttps://bbc.in/38roJFI\xa0', "Roxette singer Marie Fredriksson, who achieved global success in the 1990s with hits including 'It Must Have Been Love', dies aged 61http://bbc.in/2RDEIuo\xa0", 'Roxette singer Marie Fredriksson dies, aged 61https://bbc.in/348Pc7U\xa0', 'Ice Bucket Challenge inspiration Pete Frates dies at 34https://bbc.in/36iNjqD\xa0', "Conditions at the Sydney Cricket Ground were described as  toxic  as play continued despite smoke from bushfires. For someone like me who smokes 40 a day, it's now like smoking 80 cigarettes a day.  https://bbc.in/36lVWAJ\xa0bbccricketpic.twitter.com/CmKJH

## BBC News

In [42]:
'''Scrape titles from BBC news'''

from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
import sqlite3

url = "http://www.bbc.co.uk/news"
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

bbc_text = soup.find_all("a",class_="gs-c-promo-heading")
bbc_texts = [e.text for e in bbc_text]
print(bbc_texts)

["NZ volcano rescuer tells of 'Chernobyl'-like scene", "NZ volcano rescuer tells of 'Chernobyl'-like scene", 'Democrats set to unveil Trump impeachment charges', 'Antarctica-bound plane missing with 38 on board', 'Do apostrophes still matter?', "Canadians held for a year by China are 'resilient'", 'Video 2 minutes 10 secondsThe US city giving away free money', 'Gunman kills six in Czech hospital waiting room', 'Suu Kyi to defend Myanmar at UN genocide hearing', 'Ice Bucket Challenge inspiration dies at 34', 'Roxette singer Marie Fredriksson dies, aged 61', 'Ice Bucket Challenge inspiration dies at 34', 'Roxette singer Marie Fredriksson dies, aged 61', "Sydney 'choking' on thick smoke from bushfires", "Drugs and guns found on Juice Wrld's jet", 'Floods and power cuts hit South Africa', 'VideoOn the campaign trail with Biden', 'BBC World News TV', 'AudioBBC World Service Radio', 'VideoPlane spins on runway during emergency landing', "Dummies in police uniforms 'control' city traffic", 'V

## Table

In [43]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

'''Scrape a table(countries) from the wikipedia'''

url = "https://zh.wikipedia.org/zh-tw/ISO_3166-1"
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

table = soup.find("table", class_="wikitable sortable")

tableheader = [th.text.replace("\n","") for th in table.find_all("th")]

'''find the data of each country'''

trs = table.find_all("tr")[1:]
rows = []
for tr in trs:
    rows.append([td.text.replace("\n","").replace("\xa0","") for td in tr.find_all('td')])

#print(trs)
#print(rows)


df = pd.DataFrame(data=rows, columns=tableheader)
df.head()

,英文短名稱,二位代碼,三位代碼,數字代碼,ISO 3166-2,中文名稱,獨立主權
0,Afghanistan,AF,AFG,004,ISO 3166-2:AF,阿富汗,是
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,奧蘭,否
2,Albania,AL,ALB,008,ISO 3166-2:AL,阿爾巴尼亞,是
3,Algeria,DZ,DZA,012,ISO 3166-2:DZ,阿爾及利亞,是
4,American Samoa,AS,ASM,016,ISO 3166-2:AS,美屬薩摩亞,否
